In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
# import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import numpy as np
from time import sleep
import pandas as pd
from selenium.webdriver.common.keys import Keys

Definimos el objeto driver

In [45]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.google.com/'
driver.get(url)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


Definimos los elementos en los que se deberá hacer click o escribir durante el proceso:

In [46]:
accept_cookies_google = driver.find_element(By.XPATH, '//*[@id="L2AGLb"]')
accept_cookies_google.click()

In [47]:
buscador = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
buscador.send_keys('Indeed Iberdrola')
buscador.send_keys(Keys.ENTER)

In [48]:
search_results = driver.find_elements(By.XPATH, "//div[@class='yuRUbf']/a[@href]")
first_result = search_results[0]
first_result.click()

In [49]:
accept_cookies_indeed = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies_indeed.click()

In [50]:
page_resumen = driver.find_element(By.XPATH, '//*[@id="cmp-skip-header-desktop"]/div/ul/li[1]/a')
page_resumen.click()

In [51]:
page_source = driver.page_source
soup = bs(page_source, 'lxml')

In [52]:
nombre_empresa = soup.find("div", class_="css-1e5qoy2 e37uo190").text
nombre_empresa

'Iberdrola S.A'

In [68]:
rating_general = soup.find('span', class_='css-htn3vt e1wnkr790').text
rating_general

'3,7'

In [54]:
numero_valoraciones = soup.find("div", class_="css-r228jg eu4oa1w0").text
numero_valoraciones

'64'

In [45]:
lista_años_valoraciones = []
lista_valoraciones = []

lista_tendencia_valoraciones = soup.find_all('g', class_='css-a50thi eu4oa1w0')
for index, tendencia in enumerate(lista_tendencia_valoraciones):
    tendencia = tendencia.text
    lista_valoraciones.append(tendencia[:len(tendencia)//2])
    lista_años_valoraciones.append(tendencia[len(tendencia)//2:])

tendencia_valoraciones_2022 = soup.find('g', class_='css-je7s01 eu4oa1w0').text
lista_valoraciones.append(tendencia_valoraciones_2022[:len(tendencia_valoraciones_2022)//2])
lista_años_valoraciones.append(tendencia_valoraciones_2022[len(tendencia_valoraciones_2022)//2:])
print(lista_años_valoraciones)
print(lista_valoraciones)

['2018', '2019', '2020', '2021', '2022']
['3.00', '2.29', '3.17', '5.00', '2.75']


In [46]:
lista_categoria_valoraciones = [] 
categorias_valoraciones = soup.find_all('span', class_='css-18e8153 e1wnkr790')
for categoria in categorias_valoraciones:
    lista_categoria_valoraciones.append(categoria.text)
lista_categoria_valoraciones

['Conciliación',
 'Compensación y beneficios',
 'Estabilidad laboral/Desarrollo profesional',
 'Gestión',
 'Cultura laboral']

In [47]:
lista_puntuacion_categorias = []
puntuacion_categorias = soup.find_all('span', class_='css-1qdoj65 e1wnkr790')
for puntuacion in puntuacion_categorias:
    lista_puntuacion_categorias.append(puntuacion.text)
lista_puntuacion_categorias

['3.4', '3.1', '3.1', '3.2', '3.3']

In [49]:
page_salary = driver.find_element(By.XPATH, '//*[@id="cmp-skip-header-desktop"]/div/ul/li[4]/a')
page_salary.click()

In [50]:
page_source = driver.page_source
soup = bs(page_source, 'lxml')

In [51]:
salary_satisfaction = soup.find('div', class_='cmp-SalarySatisfactionSidebarWidgetPieChart-inside')
salary_satisfaction = salary_satisfaction.text
salary_satisfaction = salary_satisfaction[:2]
salary_satisfaction

'39'

In [109]:
def define_companies_list(file):
    companies_dataframe = pd.read_csv(file)
    companies_list = companies_dataframe.Name.values.tolist()
    return companies_list

In [110]:
def create_soup(url):
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    return soup

In [111]:
def define_review_category_list():

    soup = create_soup('https://es.indeed.com/cmp/Acciona')
    
    lista_categoria_valoraciones = [] 
    categorias_valoraciones = soup.find_all('span', class_='css-18e8153 e1wnkr790')
    
    for categoria in categorias_valoraciones:
        lista_categoria_valoraciones.append(categoria.text)
    
    return lista_categoria_valoraciones

In [112]:
def define_reviewed_years():

    soup = create_soup('https://es.indeed.com/cmp/Acciona')

    lista_años_valoraciones = []
    lista_tendencia_valoraciones = soup.find_all('g', class_='css-a50thi eu4oa1w0')
    
    for index, tendencia in enumerate(lista_tendencia_valoraciones):
        tendencia = tendencia.text
        lista_años_valoraciones.append(tendencia[len(tendencia)//2:])

    tendencia_valoraciones_2022 = soup.find('g', class_='css-je7s01 eu4oa1w0').text
    lista_años_valoraciones.append(tendencia_valoraciones_2022[len(tendencia_valoraciones_2022)//2:])
    
    return lista_años_valoraciones

In [185]:
companies_list = define_companies_list('women_presence_board_under_500_2021.csv')
lista_años_comprobacion = ['2018', '2019', '2020', '2021', '2022']
ratings_company_list = []
ratings_list = []
rating_2018_list = []
rating_2019_list = []
rating_2020_list = []
rating_2021_list = []
rating_2022_list = []
conciliacion_list = []
compensacion_y_beneficios_list = []
estabilidad_laboral_desarrollo_profesional_list = []
gestion_list = []
cultura_laboral_list = []
numero_valoraciones_list = []
satisfaccion_sueldo_list = []
count = 0

In [186]:
reviews_dictionary = {'Company' : ratings_company_list,
                    'rating' : ratings_list,
                    'ratings number' : numero_valoraciones_list,
                    # '2018' : rating_2018_list,
                    # '2019' : rating_2019_list,
                    # '2020' : rating_2020_list,
                    # '2021' : rating_2021_list,
                    # '2022' : rating_2022_list,
                    'conciliacion' : conciliacion_list,
                    'compensacion/beneficios' : compensacion_y_beneficios_list,
                    'estabilidad y desarrollo' : estabilidad_laboral_desarrollo_profesional_list,
                    'gestion' : gestion_list,
                    'cultura laboral' : cultura_laboral_list,
                    'salary satisfaction' : satisfaccion_sueldo_list
                    }

In [187]:
reviewed_category_list = ['Equilibrio vida personal/laboral', 'Salario/Beneficios', 'Estabilidad laboral/Desarrollo profesional', 'Gestión', 'Cultura laboral']

driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.google.com/'
driver.get(url)

# Se define el botón de las cookies y se aceptan las cookies de Google
accept_cookies_google = driver.find_element(By.XPATH, '//*[@id="L2AGLb"]')
accept_cookies_google.click()

for company in companies_list:

    # Se define el resultado de búsqueda como el nombre de la empresa, añadiendo 'valoraciones indeed'
    if company == 'Telefonica':
        company_search = company + ' ' + 'valoraciones indeed 65'
    else:
        company_search = company + ' ' + 'valoraciones indeed'

    # Se establece la url de inicio como google.com
    driver.get(url)

    # Se define la barra de búsqueda y se busca el nombre de la empresa
    buscador = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
    buscador.send_keys(company_search)
    buscador.send_keys(Keys.ENTER)

    # Se definen los enlaces a los resultados de búsqueda y se hace click en el primero
    search_results = driver.find_elements(By.XPATH, "//div[@class='yuRUbf']/a[@href]")
    first_result = search_results[0]
    first_result.click()

    if count == 0:
        # Se acceptan las cookies de la página de Indeed
        accept_cookies_indeed = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        sleep(2)
        accept_cookies_indeed.click()


    # Nos aseguramos de estar en la página de las valoraciones
    try:
        indeed_valoraciones = driver.find_element(By.XPATH, '//*[@id="cmp-skip-header-desktop"]/div/ul/li[3]/a')
        indeed_valoraciones.click()
    except:
        continue
    
    # Se crea la sopa de la página de valoraciones
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    try:
        keys_list = []
        values_list = []

        puntuacion_categorias = soup.find_all('a', class_='css-1cwhii6 e8ju0x51')

        for puntuacion in puntuacion_categorias:
            keys = puntuacion.text
            keys_list.append(keys[len(puntuacion)//1:])
            values = puntuacion.text
            values_list.append(values[:len(puntuacion)//1])

        diccionario_puntuacion_categorias = dict(zip(keys_list,values_list))

        for category in reviewed_category_list:
            if category not in list(diccionario_puntuacion_categorias.keys()):
                diccionario_puntuacion_categorias[category] = np.nan

        for key, value in diccionario_puntuacion_categorias.items():
            if key == 'Equilibrio vida personal/laboral':
                conciliacion_list.append(value)
            elif key == 'Salario/Beneficios':
                compensacion_y_beneficios_list.append(value)
            elif key == 'Estabilidad laboral/Desarrollo profesional':
                estabilidad_laboral_desarrollo_profesional_list.append(value)
            elif key == 'Gestión':
                gestion_list.append(value)
            elif key == 'Cultura laboral':
                cultura_laboral_list.append(value)

    except:
        conciliacion_list.append(np.nan)
        compensacion_y_beneficios_list.append(np.nan)
        estabilidad_laboral_desarrollo_profesional_list.append(np.nan)
        gestion_list.append(np.nan)
        cultura_laboral_list.append(np.nan)

    # Se accede a la página de resumen
    sleep(1)
    try:
        page_resumen = driver.find_element(By.XPATH, '//*[@id="cmp-skip-header-desktop"]/div/ul/li[1]/a')
        page_resumen.click()
    except:
        continue

    # Se crea la sopa de la página de resumen
    try:
        page_source = driver.page_source
        soup = bs(page_source, 'lxml')
    except:
        continue

    ratings_company_list.append(company)
    
    #Extracción del rating general
    try:
        rating_general = soup.find('span', class_='css-htn3vt e1wnkr790').text
        ratings_list.append(rating_general)
        print(rating_general)
    except:
        ratings_list.append(np.nan)

    # Extracción del número de valoraciones
    try:
        numero_valoraciones_totales = soup.find("div", class_="css-r228jg eu4oa1w0").text
        numero_valoraciones_list.append(numero_valoraciones_totales)
    except:
        numero_valoraciones_list.append(np.nan)

    # # Extracción de la lista de los años de los cuales tiene valoraciones y de la lista de los valores de las valoraciones
    # try:
    #     lista_años_valoraciones = []
    #     lista_valoraciones = []

    #     print('estoy sacando los años de las valoraciones')

    #     lista_tendencia_valoraciones = soup.find_all('g', class_='css-a50thi eu4oa1w0')

    #     print('estos son los años de las valoraciones: ', lista_tendencia_valoraciones)

    #     for index, tendencia in enumerate(lista_tendencia_valoraciones):
    #         tendencia = tendencia.text
    #         lista_valoraciones.append(tendencia[:len(tendencia)//2])
    #         lista_años_valoraciones.append(tendencia[len(tendencia)//2:])

    #     tendencia_valoraciones_2022 = soup.find('g', class_='css-je7s01 eu4oa1w0').text
    #     lista_valoraciones.append(tendencia_valoraciones_2022[:len(tendencia_valoraciones_2022)//2])
    #     lista_años_valoraciones.append(tendencia_valoraciones_2022[len(tendencia_valoraciones_2022)//2:])

    #     diccionario_años_valoraciones = dict(zip(lista_años_valoraciones,lista_valoraciones))

    #     for año in lista_años_comprobacion:
    #         if año not in list(diccionario_años_valoraciones.keys()):
    #             diccionario_años_valoraciones[año] = np.nan

    #     for key, value in diccionario_años_valoraciones.items():
    #         print(value)
    #         if key == '2018':
    #             rating_2018_list.append(value)
    #         elif key == '2019':
    #             rating_2019_list.append(value)
    #         elif key == '2020':
    #             rating_2020_list.append(value)
    #         elif key == '2021':
    #             rating_2021_list.append(value)
    #         elif key == '2022':
    #             rating_2022_list.append(value) 
    # except:
    #     rating_2018_list.append(np.nan)
    #     rating_2019_list.append(np.nan)
    #     rating_2020_list.append(np.nan)
    #     rating_2021_list.append(np.nan)
    #     rating_2022_list.append(np.nan)

    # print('esta es la lista 2018', rating_2018_list)
    # print('esta es la lista 2019', rating_2019_list)
    # print('esta es la lista 2020', rating_2020_list)
    # print('esta es la lista 2021', rating_2021_list)
    # print('esta es la lista 2022', rating_2022_list)

    # Se accede a la página de los sueldos
    sleep(1)
    try:
        page_salary = driver.find_element(By.XPATH, '//*[@id="cmp-skip-header-desktop"]/div/ul/li[4]/a')
        page_salary.click()
    except:
        continue

    # Se crea la sopa para la página de los sueldos
    try:
        page_source = driver.page_source
        soup = bs(page_source, 'lxml')
    except:
        continue

    # Extracción del porcentaje de satisfacción salarial
    try:
        salary_satisfaction = soup.find('div', class_='cmp-SalarySatisfactionSidebarWidgetPieChart-inside')
        salary_satisfaction = salary_satisfaction.text
        salary_satisfaction = salary_satisfaction[:2]
        satisfaccion_sueldo_list.append(salary_satisfaction)
    except:
        satisfaccion_sueldo_list.append(np.nan)
        
    count += 1
    sleep(2)
    print(company)  
    print(len(ratings_company_list))
    print(len(ratings_list)) 
    print(len(numero_valoraciones_list))
    # print(len(rating_2018_list))              
    # print(len(rating_2019_list))
    # print(len(rating_2020_list))
    # print(len(rating_2021_list))
    # print(len(rating_2022_list))
    print(len(conciliacion_list))
    print(len(compensacion_y_beneficios_list))
    print(len(estabilidad_laboral_desarrollo_profesional_list))
    print(len(gestion_list))
    print(len(cultura_laboral_list))
    print(len(satisfaccion_sueldo_list))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


4,0
Adolfo Dominguez
1
1
1
1
1
1
1
1
1
4,5
Airtificial
2
2
2
2
2
2
2
2
2
3,5
Amper
3
3
3
3
3
3
3
3
3
3,8
Arima Real Estate
4
4
4
4
4
4
4
4
4
4,4
Azkoyen
5
5
5
5
5
5
5
5
5
2,0
Bodegas Riojanas
6
6
6
6
6
6
6
6
6
3,9
Clinica Baviera
7
7
7
7
7
7
7
7
7
4,2
Codere
8
8
8
8
8
8
8
8
8
4,3
Cevasa
9
9
9
9
9
9
9
9
9
3,9
Cleo
10
10
10
10
10
10
10
10
10
3,3
Deoleo
11
11
11
11
11
11
11
11
11
4,0
DESA
12
12
12
12
12
12
12
12
12
4,3
Duro Felguera
13
13
13
13
13
13
13
13
13
3,1
Ecolumber
14
14
14
14
14
14
14
14
14
4,8
Ercros
15
15
15
15
15
15
15
15
15
4,0
GAM
16
16
16
16
16
16
16
16
16
3,9
Grupo Ecoener
17
17
17
17
17
17
17
17
17
3,8
Grupo San Jose
18
18
18
18
18
18
18
18
18
3,7
Grupo Ezentis
19
19
19
19
19
19
19
19
19
4,0
Iberpapel
20
20
20
20
20
20
20
20
20
3,8
Inmobiliaria Del Sur
21
21
21
21
21
21
21
21
21
3,5
Laboratorio Reig Jofre
22
22
22
22
22
22
22
22
22
4,4
Lar España
23
23
23
23
23
23
23
23
23
2,9
Libertas 7
24
24
24
24
24
24
24
24
24
4,2
Lingotes Especiales
25
25
25
25
25
25
25
25
25
3,7
Liw

In [96]:
ibex_35_ratings_dataframe = pd.DataFrame.from_dict(reviews_dictionary)

In [97]:
ibex_35_ratings_dataframe.to_csv('ibex_35_ratings_data.csv')

In [166]:
over_500_ratings_dataframe = pd.DataFrame.from_dict(reviews_dictionary)

In [167]:
over_500_ratings_dataframe.to_csv('over_500_ratings_data.csv')

In [188]:
under_500_ratings_dataframe = pd.DataFrame.from_dict(reviews_dictionary)

In [189]:
under_500_ratings_dataframe.to_csv('under_500_ratings_.csv')